In [10]:
from etl.dataloader import DataLoader
from utils.os_ import get_git_root
import os

import pandas as pd

# Source info to dataset

In [2]:
git_root = get_git_root(os.getcwd())
path_to_file = os.path.join(git_root, "raw")
filename = "Analytics Interview Question_mobile_new_2024.xlsx"

# Loading dataset

In [3]:
data_loader = DataLoader()
df = data_loader.load_data_xlsx_from_tab(
    path=os.path.join(path_to_file, filename), 
    sheet_name="Data")

# Raw data inspection

## data types and missing values

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 73688 entries, 0 to 73687
Data columns (total 9 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Week                         73688 non-null  object 
 1   Mobile Indicator Name        73688 non-null  object 
 2   Platform Type Name           73688 non-null  object 
 3   Super Region                 53423 non-null  object 
 4   Country Name                 73688 non-null  object 
 5   Booking Window Group         73688 non-null  object 
 6   Property Country             73688 non-null  object 
 7   Net Gross Booking Value USD  73688 non-null  float64
 8   Net Orders                   73688 non-null  int64  
dtypes: float64(1), int64(1), object(7)
memory usage: 5.1+ MB


In [6]:
df.isna().sum()

Week                               0
Mobile Indicator Name              0
Platform Type Name                 0
Super Region                   20265
Country Name                       0
Booking Window Group               0
Property Country                   0
Net Gross Booking Value USD        0
Net Orders                         0
dtype: int64

In [83]:
# sanity check: no duplicates in dimensions tuple
df_tmp = df.drop(columns=["Net Gross Booking Value USD", "Net Orders"]).copy().fillna({"Super Region": "North America"})
print(df_tmp.duplicated().sum())
df_tmp[df_tmp.duplicated(keep=False)]

0


,Week,Mobile Indicator Name,Platform Type Name,Super Region,Country Name,Booking Window Group,Property Country


## sampling and inspecting rows

In [5]:
df.sample(10)

,Week,Mobile Indicator Name,Platform Type Name,Super Region,Country Name,Booking Window Group,Property Country,Net Gross Booking Value USD,Net Orders
2285,2022-W45,Desktop,Desktop,EMEA,United Kingdom,15-30 days,Portugal,4456.5019,13
70053,2023-W48,Mobile,Mobile App,EMEA,United Kingdom,2-3 days,Vietnam,1331.4997,10
3704,2022-W45,Desktop,Desktop,NaN,US,61-90 days,Mexico,56513.8638,99
11171,2022-W46,Desktop,Desktop,LATAM,Brazil,46-60 days,Brazil,133558.6414,222
60824,2023-W47,Mobile,Mobile App,NaN,US,8-14 days,Montenegro,145.8645,1
38824,2023-W45,Mobile,Mobile App,APAC,Hong Kong,+90 days,Macau,146.4924,-1
31567,2022-W48,Mobile,Mobile Web,APAC,South Korea,8-14 days,Cyprus,245.8080,1
21982,2022-W47,Mobile,Mobile App,EMEA,Norway,2-3 days,Iceland,134.4465,1
9229,2022-W46,Desktop,Desktop,APAC,South Korea,46-60 days,Palau,-352.4880,-1
33554,2022-W48,Mobile,Mobile Web,NaN,US,8-14 days,South Korea,-338.1200,3


## checking unique values

In [12]:
def return_series_of_unique_values(s: pd.Series):
    return pd.Series(s.unique()).sort_values()

In [13]:
weeks = return_series_of_unique_values(df["Week"])
weeks

0    2022-W45
1    2022-W46
2    2022-W47
3    2022-W48
4    2023-W45
5    2023-W46
6    2023-W47
7    2023-W48
dtype: object

In [14]:
devices = weeks = return_series_of_unique_values(df["Mobile Indicator Name"])
devices

0    Desktop
1     Mobile
dtype: object

In [15]:
platforms = weeks = return_series_of_unique_values(df["Platform Type Name"])
platforms

0       Desktop
1    Mobile App
2    Mobile Web
dtype: object

In [16]:
# sanity check
df[["Mobile Indicator Name", "Platform Type Name"]].drop_duplicates()

,Mobile Indicator Name,Platform Type Name
0,Desktop,Desktop
4181,Mobile,Mobile App
6141,Mobile,Mobile Web


In [36]:
df[["Mobile Indicator Name", "Platform Type Name"]].value_counts()

Mobile Indicator Name  Platform Type Name
Desktop                Desktop               35048
Mobile                 Mobile Web            19852
                       Mobile App            18788
Name: count, dtype: int64

In [17]:
super_regions = return_series_of_unique_values(df["Super Region"])
super_regions

0     APAC
1     EMEA
2    LATAM
3      NaN
dtype: object

In [35]:
df["Super Region"].fillna("NA").value_counts()

Super Region
APAC     23805
EMEA     23477
NA       20265
LATAM     6141
Name: count, dtype: int64

In [18]:
countries_missing_super_region = return_series_of_unique_values(df[df["Super Region"].isna()]["Country Name"])
countries_missing_super_region

0    US
dtype: object

In [20]:
countries = return_series_of_unique_values(df["Country Name"]).reset_index(drop=True)
countries

0         Australia
1            Brazil
2         Hong Kong
3            Norway
4       South Korea
5                US
6    United Kingdom
dtype: object

In [23]:
# sanity check
df[["Super Region", "Country Name"]].drop_duplicates().reset_index(drop=True).sort_values(by=["Super Region", "Country Name"])

,Super Region,Country Name
1,APAC,Australia
2,APAC,Hong Kong
0,APAC,South Korea
4,EMEA,Norway
3,EMEA,United Kingdom
5,LATAM,Brazil
6,NaN,US


In [25]:
destination_country = return_series_of_unique_values(df["Property Country"]).reset_index(drop=True)
destination_country

0             Albania
1             Algeria
2      American Samoa
3             Andorra
4              Angola
            ...      
201           Vanuatu
202         Venezuela
203           Vietnam
204            Zambia
205          Zimbabwe
Length: 206, dtype: object

In [31]:
df[["Super Region", "Property Country"]]\
    .fillna({"Super Region": "North America"})\
    .groupby("Super Region")["Property Country"]\
    .value_counts().groupby(level=0).head(6)

Super Region   Property Country        
APAC           Thailand                    646
               United States of America    642
               Japan                       640
               Australia                   637
               United Kingdom              616
               Singapore                   615
EMEA           United States of America    434
               United Kingdom              433
               Spain & Canary Islands      431
               Thailand                    430
               Germany                     424
               France                      411
LATAM          Brazil                      216
               United States of America    216
               Italy                       202
               Argentina                   200
               Uruguay                     195
               France                      194
North America  United States of America    238
               Canada                      221
               Mexic

In [32]:
booking_window_group = return_series_of_unique_values(df["Booking Window Group"]).reset_index(drop=True)
booking_window_group

0      +90 days
1      0-1 days
2    15-30 days
3      2-3 days
4    31-45 days
5      4-7 days
6    46-60 days
7    61-90 days
8     8-14 days
9     Post Book
dtype: object

In [34]:
df["Booking Window Group"].value_counts()

Booking Window Group
0-1 days      9331
15-30 days    9080
+90 days      8529
8-14 days     8243
4-7 days      8175
31-45 days    8084
2-3 days      8018
46-60 days    7131
61-90 days    7057
Post Book       40
Name: count, dtype: int64

In [38]:
df[["Net Gross Booking Value USD", "Net Orders"]].describe()

,Net Gross Booking Value USD,Net Orders
count,7.368800e+04,73688.000000
mean,1.274363e+04,47.954606
std,2.072294e+05,991.128900
min,-9.689693e+04,-50.000000
25%,2.158794e+02,1.000000
50%,8.575607e+02,2.000000
75%,3.198414e+03,8.000000
max,1.166149e+07,63230.000000


In [51]:
df[df["Net Gross Booking Value USD"] < 0].shape[0]

6200

In [41]:
df[df["Net Gross Booking Value USD"] < 0].shape[0] / df.shape[0]

0.0841385300184562

In [44]:
df[df["Net Gross Booking Value USD"] < 0]["Net Orders"].describe()

count    6200.000000
mean       -0.031452
std         3.932132
min       -50.000000
25%        -1.000000
50%        -1.000000
75%         0.000000
max       110.000000
Name: Net Orders, dtype: float64

In [50]:
df[(df["Net Gross Booking Value USD"] < 0) & (df["Net Orders"] > 0)].sort_values(by="Net Orders", ascending=False)

,Week,Mobile Indicator Name,Platform Type Name,Super Region,Country Name,Booking Window Group,Property Country,Net Gross Booking Value USD,Net Orders
3440,2022-W45,Desktop,Desktop,NaN,US,0-1 days,Germany,-21707.4969,110
3275,2022-W45,Desktop,Desktop,NaN,US,0-1 days,China,-13937.1861,91
3418,2022-W45,Desktop,Desktop,NaN,US,0-1 days,France,-66789.2746,90
3974,2022-W45,Desktop,Desktop,NaN,US,0-1 days,Spain & Canary Islands,-5504.0696,74
20188,2022-W47,Desktop,Desktop,NaN,US,0-1 days,China,-32542.9080,69
...,...,...,...,...,...,...,...,...,...
38145,2023-W45,Desktop,Desktop,NaN,US,8-14 days,St. Kitts and Nevis,-145.6400,1
38168,2023-W45,Desktop,Desktop,NaN,US,4-7 days,St. Vincent and the Grenadines,-96.0000,1
38215,2023-W45,Desktop,Desktop,NaN,US,61-90 days,Tanzania,-610.0000,1
38381,2023-W45,Mobile,Mobile App,APAC,Hong Kong,61-90 days,Australia,-92.6899,1


In [57]:
weekly_agg = df[["Week", "Super Region", "Net Gross Booking Value USD", "Net Orders"]]\
    .fillna({"Super Region": "North America"})\
    .groupby(["Week", "Super Region"])\
    .agg({"Net Gross Booking Value USD": "sum", "Net Orders": "sum"})\
    .sort_values(by=["Week", "Super Region"], ascending=[True, True])

weekly_agg["avg_order_ticket"] = (weekly_agg["Net Gross Booking Value USD"] / weekly_agg["Net Orders"]).round(2)
weekly_agg["Net Gross Booking Value USD"] *= 1e-6
weekly_agg["Net Orders"] *= 1e-3

weekly_agg = weekly_agg.round(2)
weekly_agg = weekly_agg.sort_index()
weekly_agg

Net Gross Booking Value USD  Net Orders  \
Week     Super Region                                             
2022-W45 APAC                                 25.42       31.21   
         EMEA                                 16.61       47.55   
         LATAM                                 4.02        9.04   
         North America                        81.26      316.58   
2022-W46 APAC                                 11.71       35.10   
         EMEA                                 17.06       51.42   
         LATAM                                 4.18        9.44   
         North America                        81.63      317.36   
2022-W47 APAC                                  9.74       30.78   
         EMEA                                 14.82       45.47   
         LATAM                                 3.92        8.88   
         North America                        70.76      288.57   
2022-W48 APAC                                 10.20       31.14   
         EMEA                                 14.32       44.38   
         LATAM                                 4.39        9.31   
         North America                        61.99      260.08   
2023-W45 APAC                                 14.94       51.96   
         EMEA                                 20.64       68.95   
         LATAM                                 4.40       14.88   
         North America                        97.63      388.24   
2023-W46 APAC                                 14.61       51.92   
         EMEA                                 20.59       67.26   
         LATAM                                 4.29       15.13   
         North America                        95.87      385.34   
2023-W47 APAC                                 13.37       49.27   
         EMEA                                 17.77       61.58   
         LATAM                                 3.53       12.38   
         North America                        88.15      378.09   
2023-W48 APAC                                 14.74       52.83   
         EMEA                                 17.64       61.84   
         LATAM                                 4.10       13.56   
         North America                        74.76      324.14   

                        avg_order_ticket  
Week     Super Region                     
2022-W45 APAC                     814.59  
         EMEA                     349.39  
         LATAM                    444.80  
         North America            256.67  
2022-W46 APAC                     333.58  
         EMEA                     331.85  
         LATAM                    442.41  
         North America            257.22  
2022-W47 APAC                     316.37  
         EMEA                     325.92  
         LATAM                    441.48  
         North America            245.19  
2022-W48 APAC                     327.58  
         EMEA                     322.76  
         LATAM                    471.25  
         North America            238.36  
2023-W45 APAC                     287.47  
         EMEA                     299.32  
         LATAM                    295.65  
         North America            251.47  
2023-W46 APAC                     281.45  
         EMEA                     306.17  
         LATAM                    283.73  
         North America            248.80  
2023-W47 APAC                     271.30  
         EMEA                     288.53  
         LATAM                    284.74  
         North America            233.15  
2023-W48 APAC                     278.99  
         EMEA                     285.17  
         LATAM                    302.14  
         North America            230.65

In [93]:
# filter "Week" that contains year 2022 in string
# filter super region = "North America" 
week_str_filter = "2023"
region_str_filter = ["North America"]

weekly_agg_filtered = weekly_agg[
    (weekly_agg.index.get_level_values(0).str.contains(week_str_filter)) & 
    (weekly_agg.index.get_level_values(1).isin(region_str_filter))]
print(weekly_agg_filtered.mean())
weekly_agg_filtered

Net Gross Booking Value USD     89.1025
Net Orders                     368.9525
avg_order_ticket               241.0175
dtype: float64


,,Net Gross Booking Value USD,Net Orders,avg_order_ticket
Week,Super Region,,,
2023-W45,North America,97.63,388.24,251.47
2023-W46,North America,95.87,385.34,248.80
2023-W47,North America,88.15,378.09,233.15
2023-W48,North America,74.76,324.14,230.65


## notes about columns

### dimensions

- Week: 
  - dates range from W45 to W48 (Nov)
  - years 2022 and 2023
  - can create a method to decompose string and get year/week date formats
- Mobile indicator:
  - either `Desktop` or `Mobile`
- Platform types:
  - Destkop (35k)
  - App (Mobile) 19k
  - Web (Mobile) 18k
- Super Region
  - APAC, EMEA, LATAM
  - many missing (`NaN`) values: all of them have country `US`
  - LATAM is undersampled, the others look uniform
- Countries
  - Total: 7
  - 'Australia', 'Brazil', 'Hong Kong', 'Norway', 'South Korea', 'US', 'United Kingdom'
  - 2 APAC, 3 EMEA, 1 LATAM, 1 US/NA
- Property country:
  - 207 unique
  - US in top 5 destination of all super regions
  - Thailand and UK top 5 for both APAC and EMEA
- Booking Window Group
  - string is a bit messy
  - represents bins of time-length (in days) between booking and check in
  - except for 1 category, looks uniformly distributed in sampling
  - "Post book" value: 
    - probably covers any booking activity that occurs post the initial check-in, 
    - contrasting with the other categories which all refer to the period before check-in
    - can fill "Post book" with -1 if it helps to turn this into numeric
    - only 40 rows, might as well discard

### facts / aggregated numerical values

Values are aggregated on the following dimensions tuple:  
`(Week, Device Type, Platform, Super Region, Country Name, Property Country, Booking Window Group)`


- Net Gross Booking Value USD
  - Definition: The total $ amount that customers pay to Hotels.com for their hotel reservation. The value of cancelled bookings is removed
  - some values are very negative (8.4% of rows)
    - from the definition, cancellations are removed (deducted), so it could explain it
    - 1319 out of 6200 have net orders positive
      - could this be result of discounts applied? 
      - by reading the definition ("amount that customers pay to") doesn't make much sense
  - US has the largest total bookings but lower avg. tickets
  - LATAM has very low total bookings but larger avg. ticket
  - APAC 2022 first week has double the avg. bookings and double avg. ticket, looks weird
- Net Orders
  - Definition: The total number of hotel bookings made - the number of bookings cancelled
  - some negative values 
  - negatives are fine because, by definition, bookings cancelled are subtracted

Typical Values per Super Region:

APAC*:
  - Bookings: $10$M - $14$M
  - Orders: $32$k - $51$k
  - Ticket: $\$320$ - $\$280$

EMEA:
  - Bookings: $15$M - $19$M
  - Orders: $47$k - $64$k
  - Ticket: $\$332$ - $\$294$

 LATAM (Brazil):
  - Bookings: $4$M - $4$M
  - Orders: $9$k - $14$k
  - Ticket: $\$450$ - $\$291$

North America (US):
  - Bookings: $73$M - $89$M
  - Orders: $295$k - $368$k
  - Ticket: $\$249$ - $\$241$

* if taking actual bookings first week 2022, it is 14M and avg. ticket $448